In [ ]:
!pip install SimpleITK
!pip install pyradiomics

In [ ]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import time
import SimpleITK as sitk
from matplotlib.pyplot import cm
from skimage import measure
from skimage.measure import regionprops
from radiomics import featureextractor
import six
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Code

In [ ]:
img = sitk.ReadImage(f'./ProstateX/ProstateX-0000/T2W.nii.gz')

In [ ]:
# Reading the prostate mask and lesion annotation
pm = sitk.ReadImage("./ProstateX/ProstateX-0000/PM.nii.gz")
ls = sitk.ReadImage("./ProstateX/ProstateX-0000/LS1.nii.gz")

In [ ]:
# pyradiomics feature extractor requires some initial parameters to be set

params = {}
params['normalize'] = True
params['binWidth'] = 256
params['verbose'] = True

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeatureExtractor(**params) 

# First disablign all features and enabling only glcm features
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('glcm')

featureVector = extractor.execute(img, pm)
print(list(featureVector.keys()))
for featureName in featureVector.keys():
  print(f"Computed {featureName}: {featureVector[featureName]}")

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass', 'original_glcm_Autocorrelation', 'original_glcm_ClusterProminence', 'original_glcm_ClusterShade', 'original_glcm_ClusterTendency', 'original_glcm_Contrast', 'original_glcm_C

In [ ]:
rootdir = '/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX'
img_dict = {}
ls_dict = {}
fieldnames = ['patient_id', 'ls/pm']
fieldnames.extend(list(featureVector.keys()))
print(fieldnames)
with open('features.csv', mode='w') as features:
    writer = csv.DictWriter(features, fieldnames = fieldnames)
    writer.writeheader()
    for subdir, dirs, files in os.walk(rootdir):
        subpath = subdir.split('/')
        if subpath[-1] not in 'ProstateX':
            patient_name = subpath[-1]
            img_dict[patient_name] = {}
            img = sitk.ReadImage(subdir + '/T2W.nii.gz')
            ls = sitk.ReadImage(subdir + '/LS1.nii.gz')
            pm = sitk.ReadImage(subdir+'/PM.nii.gz')
            adc = sitk.ReadImage(subdir+'/ADC.nii.gz')
            featVecLs = extractor.execute(img, ls)
            featVecPm = extractor.execute(img, pm)
            img_dict[patient_name]['image'] = img
            img_dict[patient_name]['lesion'] = ls
            img_dict[patient_name]['mask'] = pm
            img_dict[patient_name]['diff'] = adc
            featVecLs['patient_id'] = patient_name
            featVecLs['ls/pm'] = 'ls'
            featVecPm['patient_id'] = patient_name
            featVecPm['ls/pm'] = 'pm'
            writer.writerow(featVecLs)
            writer.writerow(featVecPm)





        





['patient_id', 'ls/pm', 'diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy', 'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet', 'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings', 'diagnostics_Configuration_EnabledImageTypes', 'diagnostics_Image-original_Hash', 'diagnostics_Image-original_Dimensionality', 'diagnostics_Image-original_Spacing', 'diagnostics_Image-original_Size', 'diagnostics_Image-original_Mean', 'diagnostics_Image-original_Minimum', 'diagnostics_Image-original_Maximum', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_Spacing', 'diagnostics_Mask-original_Size', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_VoxelNum', 'diagnostics_Mask-original_VolumeNum', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass', 'original_glcm_Autocorrelation', 'original_glcm_ClusterProminence', 'original_glcm_ClusterShade', 'original_glcm_ClusterTendency', 'original_glcm_Cont

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX/ProstateX-0001


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX/ProstateX-0002


...

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX/ProstateX-0191


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


/content/drive/MyDrive/Uni/Sem 8 - Spring 21/EBME 461 - Biomedical Image Processing and Analysis/Project/ProstateX/ProstateX-0177


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


OrderedDict([('diagnostics_Versions_PyRadiomics', 'v3.0.1'), ('diagnostics_Versions_Numpy', '1.19.5'), ('diagnostics_Versions_SimpleITK', '2.0.2'), ('diagnostics_Versions_PyWavelet', '1.1.1'), ('diagnostics_Versions_Python', '3.7.10'), ('diagnostics_Configuration_Settings', {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 256, 'verbose': True}), ('diagnostics_Configuration_EnabledImageTypes', {'Original': {}}), ('diagnostics_Image-original_Hash', '6ca828fd5843ea62d8e6e412091e9749b2ab60b3'), ('diagnostics_Image-original_Dimensionality', '3D'), ('diagnostics_Image-original_Spacing', (0.5, 0.5, 3.000002145767212)), ('diagnostics_Image-original_Size', (384, 384, 19)), ('diagnostics_Image-original_Me